In [ ]:
from google.colab import drive

# The following command will prompt a URL for you to click and obtain the
# authorization code

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from pathlib import Path

DATA = Path("/content/drive/My Drive/EECS 6893/Project/data")

In [ ]:
!pip install dash

     |████████████████████████████████| 7.3 MB 8.7 MB/s 
     |████████████████████████████████| 26.5 MB 1.5 MB/s 
     |████████████████████████████████| 357 kB 40.1 MB/s 
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=c93387d46876f015d13f28de441952b9521fd703b1cd1ab6394edd80643260c1
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=854e15cf091f359a944aae94e385f859c8f335b0342e4514351712084e5780f7
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=984515a754ab02c748625d1cd5fbe2fa68904bca27dd61204792fb0dbcc87a74
  Stored in directory: /root/.cache/pip/wheels/85/5d/4e/7c276b57992951dbe770bf5caad6448d0539c510663aef

In [ ]:
import dash
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
import pandas as pd
import datetime
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:
crypto_file = "crypto_close.csv"
data = pd.read_csv(DATA / crypto_file)

In [ ]:
base = datetime.date.fromisoformat('2021-09-21')
date_list = [base - datetime.timedelta(days=x) for x in range(366)]
data['Date'] = date_list[::-1]

In [ ]:
app = dash.Dash(__name__)

In [ ]:
data.head()

,eth,bit,btc,bnc,eos,ethc,ltc,mon,tron,ste,car,iota,maker,doge,Date
0,370.498571,10913.255288,225.996000,26.04485,2.669025,5.409800,47.010500,92.549667,0.026906,0.076838,0.089254,0.26240,474.821575,0.002775,2020-09-21
1,340.848571,10429.791250,212.472500,23.11265,2.507040,5.352625,43.203571,89.250000,0.024963,0.069972,0.080095,0.24070,452.825000,0.002602,2020-09-22
2,344.161689,10531.087143,219.688333,23.98805,2.570775,5.285133,44.474200,92.570000,0.024992,0.071571,0.081466,0.24375,461.136667,0.002641,2020-09-23
3,320.322857,10238.671250,207.760000,22.79610,2.440820,5.315600,42.980167,87.100000,0.024847,0.067351,0.076566,0.23280,435.175000,0.002545,2020-09-24
4,349.055179,10741.417762,216.208000,24.56370,2.533050,5.278200,44.935800,91.840000,0.026278,0.070404,0.082650,0.24680,485.255867,0.002629,2020-09-25


In [ ]:
data_bar = {'crypto_assets' : ['bit','btc','eth','bnc','eos','ethc','ltc','mon','tron','ste','car','iota','maker','doge'],
            'weight':[1/14,1/14,1/14,1/14,1/14,1/14,1/14,1/14,1/14,1/14,1/14,1/14,1/14,1/14,]}

In [ ]:
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

from rpy2.robjects.conversion import localconverter

In [ ]:
with localconverter(ro.default_converter + pandas2ri.converter):
  r_crypto = ro.conversion.py2rpy(data.drop(['Date'],axis=1))

In [ ]:
data.head()

,eth,bit,btc,bnc,eos,ethc,ltc,mon,tron,ste,car,iota,maker,doge,Date
0,370.498571,10913.255288,225.996000,26.04485,2.669025,5.409800,47.010500,92.549667,0.026906,0.076838,0.089254,0.26240,474.821575,0.002775,2020-09-21
1,340.848571,10429.791250,212.472500,23.11265,2.507040,5.352625,43.203571,89.250000,0.024963,0.069972,0.080095,0.24070,452.825000,0.002602,2020-09-22
2,344.161689,10531.087143,219.688333,23.98805,2.570775,5.285133,44.474200,92.570000,0.024992,0.071571,0.081466,0.24375,461.136667,0.002641,2020-09-23
3,320.322857,10238.671250,207.760000,22.79610,2.440820,5.315600,42.980167,87.100000,0.024847,0.067351,0.076566,0.23280,435.175000,0.002545,2020-09-24
4,349.055179,10741.417762,216.208000,24.56370,2.533050,5.278200,44.935800,91.840000,0.026278,0.070404,0.082650,0.24680,485.255867,0.002629,2020-09-25


In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R -w 5 -h 5
install.packages('quantmod')
install.packages('readxl')
install.packages('quadprog')
install.packages("PerformanceAnalytics")
install.packages("boot")
install.packages("sjstats")

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘xts’, ‘zoo’, ‘TTR’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/xts_0.12.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 517875 bytes (505 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[

In [ ]:
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
robjects.r(''' multiply <- function(x, y,crypto) { 
library(quantmod)
library(readxl)
library(quadprog)
library(PerformanceAnalytics)
library(boot)
library(sjstats)
returns = 100*(data.frame(diff(as.matrix(crypto))))/(head(crypto,-1))
mean_vect = colMeans(returns)
cov_mat = cov(returns)
sd_vect = sqrt(diag(cov_mat))
M = length(mean_vect)
muP = seq(0.45,2.31,length=300)
sdP = muP
weights = matrix(0,nrow=300,ncol=M)
Amat = cbind(rep(1,M),mean_vect,diag(1,nrow=M),diag(1,nrow=M))

for (i in 1:length(muP))
{
  result =
    solve.QP(Dmat=cov_mat,dvec=rep(0,M), Amat=Amat,
             c(1,muP[i],rep(-0.0001,M),rep(-6,M)), meq=2)
  sdP[i] = sqrt(2*result$value)
  weights[i,] = result$solution
}
mvpind <- sdP==min(sdP)
mvp_weights <- weights[mvpind]
mup_mvp <- muP[mvpind]

investment = x
mvp_weights <- round(mvp_weights,5)
mvp_return <- rowSums(as.matrix(returns)%*%diag(mvp_weights))
q = quantile(0.01*mvp_return,0.05)
VaR_mvp = q*-investment # Value at risk
IEVaR = ((0.01*mvp_return)<q)
ESMVP = -investment*sum(0.01*mvp_return*IEVaR)/sum(IEVaR)

mufree = 0.45/30
sharpe =( muP-mufree)/sdP
tan_ind = (sharpe == max(sharpe))

target_ret_weights = weights[abs(muP-y) == min(abs(muP-y))]
target_ret_weights <- round(target_ret_weights,5)
target_return <- rowSums(as.matrix(returns)%*%diag(target_ret_weights))
VaR_target = -x*VaR(0.01*target_return, method="historical")
retu = c(VaR_target,target_ret_weights)
 return(retu)}''')

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x7f3ce5ada730> [RTYPES.CLOSXP]
R classes: ('function',)

In [ ]:
multiply = robjects.r['multiply']
res = multiply(int(1000),int(0.5),r_crypto)

R[write to console]: Loading required package: xts

R[write to console]: Loading required package: zoo

R[write to console]: 
Attaching package: ‘zoo’


R[write to console]: The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


R[write to console]: Loading required package: TTR

R[write to console]: Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

R[write to console]: 
Attaching package: ‘PerformanceAnalytics’


R[write to console]: The following object is masked from ‘package:graphics’:

    legend




In [ ]:
import numpy as np
xwe = np.round(np.asarray(res),2)

In [ ]:
#len(data_bar['weight'])
sum(xwe[1:])

1.0

In [ ]:
str(type(3))

"<class 'int'>"

In [ ]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1(children="Crypto Currency management",),
    html.P(children="Select crypto asset",),
    dcc.Dropdown(
        id='demo-dropdown',
        options=[
                {'label': 'Bitcoin', 'value': 'bit'},
                {'label': 'Ethereum', 'value': 'eth'},
                {'label': 'Bitcoin cash', 'value': 'btc'},
                {'label': 'EOS.IO', 'value': 'eos'},
                {'label': 'Binanace coin', 'value': 'bnc'},
                {'label': 'Ethereum classic', 'value': 'ethc'},
                {'label': 'Litecoin', 'value': 'ltc'},
                {'label': 'Monero', 'value': 'mon'},
                {'label': 'Tron', 'value': 'tron'},
                {'label': 'Stellar', 'value': 'ste'},
                {'label': 'Cardano', 'value': 'car'},
                {'label': 'IOTA', 'value': 'iota'},
                {'label': 'maker', 'value': 'maker'},
                {'label': 'Doge coin', 'value': 'doge'}
            ],
            value = 'bit'
    ),
    dcc.Graph(id='dd-output-container'),
    html.P(children="Portfolio!!! How much to invest in USD in next month",),
    dcc.Input(id="inv", type="number",placeholder="1000"),
    html.P(children="Target ROI per day",),
    dcc.Input(id="period", type="number",placeholder="0.5"),
    html.Div(id='VaR',),
    html.H2(children="Portfolio weights for minimum risk",),
    dcc.Graph(id='dd-output-container2'),
    
    
])

# def portfolio(inv,mu):
#   res = multiply(inv,mu,r_crypto)
#   res = numpy.asarray(res)
#   return res

robjects.r(''' multiply <- function(x, y,crypto) { 
library(quantmod)
library(readxl)
library(quadprog)
library(PerformanceAnalytics)
library(boot)
library(sjstats)
returns = 100*(data.frame(diff(as.matrix(crypto))))/(head(crypto,-1))
mean_vect = colMeans(returns)
cov_mat = cov(returns)
sd_vect = sqrt(diag(cov_mat))
M = length(mean_vect)
muP = seq(0.45,2.31,length=300)
sdP = muP
weights = matrix(0,nrow=300,ncol=M)
Amat = cbind(rep(1,M),mean_vect,diag(1,nrow=M),diag(1,nrow=M))

for (i in 1:length(muP))
{
  result =
    solve.QP(Dmat=cov_mat,dvec=rep(0,M), Amat=Amat,
             c(1,muP[i],rep(-0.0001,M),rep(-6,M)), meq=2)
  sdP[i] = sqrt(2*result$value)
  weights[i,] = result$solution
}
mvpind <- sdP==min(sdP)
mvp_weights <- weights[mvpind]
mup_mvp <- muP[mvpind]

investment = x
mvp_weights <- round(mvp_weights,5)
mvp_return <- rowSums(as.matrix(returns)%*%diag(mvp_weights))
q = quantile(0.01*mvp_return,0.05)
VaR_mvp = q*-investment # Value at risk
IEVaR = ((0.01*mvp_return)<q)
ESMVP = -investment*sum(0.01*mvp_return*IEVaR)/sum(IEVaR)

mufree = 0.45/30
sharpe =( muP-mufree)/sdP
tan_ind = (sharpe == max(sharpe))

target_ret_weights = weights[abs(muP-y) == min(abs(muP-y))]
target_ret_weights <- round(target_ret_weights,5)
target_return <- rowSums(as.matrix(returns)%*%diag(target_ret_weights))
VaR_target = -x*VaR(0.01*target_return, method="historical")
retu = c(VaR_target,target_ret_weights)
 return(retu)}''')

with localconverter(ro.default_converter + pandas2ri.converter):
  r_crypto = ro.conversion.py2rpy(data)

@app.callback(
    Output('dd-output-container', 'figure'),
    Output('dd-output-container2', 'figure'),
    Output('VaR','children'),
    Input('demo-dropdown', 'value'),
    Input('inv', 'value'),
    Input('period', 'value'),
)
def update_output(curr,inv,period):
    fig1 = px.line(data, x='Date', y=curr)
    robjects.r(''' multiply <- function(x, y,crypto) { 
    library(quantmod)
    library(readxl)
    library(quadprog)
    library(PerformanceAnalytics)
    library(boot)
    library(sjstats)
    returns = 100*(data.frame(diff(as.matrix(crypto))))/(head(crypto,-1))
    mean_vect = colMeans(returns)
    cov_mat = cov(returns)
    sd_vect = sqrt(diag(cov_mat))
    M = length(mean_vect)
    muP = seq(0.45,2.31,length=300)
    sdP = muP
    weights = matrix(0,nrow=300,ncol=M)
    Amat = cbind(rep(1,M),mean_vect,diag(1,nrow=M),diag(1,nrow=M))

    for (i in 1:length(muP))
    {
      result =
        solve.QP(Dmat=cov_mat,dvec=rep(0,M), Amat=Amat,
                c(1,muP[i],rep(-0.0001,M),rep(-6,M)), meq=2)
      sdP[i] = sqrt(2*result$value)
      weights[i,] = result$solution
    }
    mvpind <- sdP==min(sdP)
    mvp_weights <- weights[mvpind]
    mup_mvp <- muP[mvpind]

    investment = x
    mvp_weights <- round(mvp_weights,5)
    mvp_return <- rowSums(as.matrix(returns)%*%diag(mvp_weights))
    q = quantile(0.01*mvp_return,0.05)
    VaR_mvp = q*-investment # Value at risk
    IEVaR = ((0.01*mvp_return)<q)
    ESMVP = -investment*sum(0.01*mvp_return*IEVaR)/sum(IEVaR)

    mufree = 0.45/30
    sharpe =( muP-mufree)/sdP
    tan_ind = (sharpe == max(sharpe))

    target_ret_weights = weights[abs(muP-y) == min(abs(muP-y))]
    target_ret_weights <- round(target_ret_weights,5)
    target_return <- rowSums(as.matrix(returns)%*%diag(target_ret_weights))
    VaR_target = -x*VaR(0.01*target_return, method="historical")
    retu = c(VaR_target,target_ret_weights)
    return(retu)}''')
    with localconverter(ro.default_converter + pandas2ri.converter):
      r_crypto = ro.conversion.py2rpy(data.drop(['Date'],axis=1))
    try:
      temp = multiply(int(inv),int(period),r_crypto)
      temp = np.round(np.asarray(temp),2)
      data_bar['weight'] = temp[1:]
    except:
      temp = [0]
    x = 'The Value at Risk at 0.05 alpha is ' + str(temp[0]) + ' dollors'
    fig2 = px.bar(data_bar, x='crypto_assets', y='weight')
    return fig1,fig2,x


# if __name__ == '__main__':
#     app.run_server()
app.run_server(mode='inline')

<IPython.core.display.Javascript object>

In [ ]:
!pip install jupyter-dash